<a href="https://colab.research.google.com/github/Rishav-hub/Auto-Gluon/blob/main/03_AutoGluon_for_Tabular_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation
AutoGluon requires Python version 3.7, 3.8, or 3.9 [link text](https://auto.gluon.ai/stable/index.html#installation)


In [3]:
# pip3 install -U pip
# pip3 install -U setuptools wheel
!pip3 install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached autogluon-0.4.2-py3-none-any.whl (9.5 kB)
  Using cached autogluon.features-0.4.2-py3-none-any.whl (59 kB)
  Using cached autogluon.tabular-0.4.2-py3-none-any.whl (267 kB)
  Using cached autogluon.text-0.4.2-py3-none-any.whl (162 kB)
  Using cached autogluon.core-0.4.2-py3-none-any.whl (189 kB)
  Using cached autogluon.vision-0.4.2-py3-none-any.whl (48 kB)
  Using cached boto3-1.24.11-py3-none-any.whl (132 kB)
  Using cached distributed-2021.11.2-py3-none-any.whl (802 kB)
  Using cached dask-2021.11.2-py3-none-any.whl (1.0 MB)
  Using cached protobuf-3.20.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Using cached autogluon.common-0.4.2-py3-none-any.whl (37 kB)
  Using cached ray-1.10.0-cp37-cp37m-manylinux2014_x86_64.whl (59.6 MB)
  Using cached psutil-5.8.0-cp37-cp37m-manylinux2010_x86_64.whl (296 kB)
  Using cached lightgbm-3.3.2-py3-none-manylinux1_x

In [1]:
!pip3 install "torch>=1.0,<1.11+cu113" -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [2]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.device_count())  # Should be > 0
print(torch.__version__)

True
1
1.10.2+cu102


## Implementation

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor

## Classification

In [5]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [6]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object


## AutoGluon to train multiple models

In [ ]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

## Prediction

In [8]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


## Predict and Evaluate

In [9]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8374449790152523
Evaluations on test data:
{
    "accuracy": 0.8374449790152523,
    "balanced_accuracy": 0.7430558394221018,
    "mcc": 0.5243657567117436,
    "f1": 0.621904761904762,
    "precision": 0.69394261424017,
    "recall": 0.5634167385677308
}


Predictions:  
 0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


## Check Leaderboard

In [10]:
predictor.leaderboard(test_data, silent=True)


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842973,0.84,0.316342,0.105665,0.738348,0.316342,0.105665,0.738348,1,True,5
1,CatBoost,0.842461,0.85,0.059466,0.008585,1.286446,0.059466,0.008585,1.286446,1,True,7
2,RandomForestEntr,0.841130,0.83,0.322081,0.105686,0.619496,0.322081,0.105686,0.619496,1,True,6
3,LightGBM,0.839799,0.85,0.136862,0.007873,0.194438,0.136862,0.007873,0.194438,1,True,4
4,XGBoost,0.837445,0.87,0.062397,0.006068,0.244396,0.062397,0.006068,0.244396,1,True,11
5,WeightedEnsemble_L2,0.837445,0.87,0.065353,0.006566,0.541433,0.002956,0.000498,0.297037,2,True,14
6,LightGBMXT,0.836421,0.83,0.086246,0.007179,2.477554,0.086246,0.007179,2.477554,1,True,3
7,ExtraTreesGini,0.833453,0.82,0.540198,0.105908,0.624001,0.540198,0.105908,0.624001,1,True,8
8,ExtraTreesEntr,0.832839,0.81,0.576187,0.105532,0.617077,0.576187,0.105532,0.617077,1,True,9
9,LightGBMLarge,0.828949,0.83,0.124668,0.007619,0.389290,0.124668,0.007619,0.389290,1,True,13


## Regression 

In [11]:
import pandas as pd
import seaborn as sns
df = sns.load_dataset('mpg')
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


## Split Dataset

In [13]:
num = int(0.9*df.shape[0])
train_data = df[:num]
print(train_data.shape)
test_data = df[num:]
print(test_data.shape)

(358, 9)
(40, 9)


In [17]:
mpg_column = 'mpg'
print("Summary of age variable: \n", train_data[mpg_column].describe())

Summary of age variable: 
 count    358.000000
mean      22.762570
std        7.650818
min        9.000000
25%       16.500000
50%       21.500000
75%       28.000000
max       46.600000
Name: mpg, dtype: float64


## Train Regression

In [19]:
predictor_mpg = TabularPredictor(label=mpg_column, path="agModels-predictMpg").fit(train_data, time_limit=60)
performance = predictor_mpg.evaluate(test_data)

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "agModels-predictMpg/"
AutoGluon Version:  0.4.2
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    358
Train Data Columns: 8
Label Column: mpg
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (46.6, 9.0, 22.76257, 7.65082)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11530.85 MB
	Train Data (Original)  Memory Usage: 0.07 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specif

## Regression Prediction

In [16]:
predictor_mpg.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-3.850339,-2.317559,0.052176,0.031579,0.291274,0.052176,0.031579,0.291274,1,True,3
1,CatBoost,-3.992764,-2.186351,0.006219,0.004123,1.036431,0.006219,0.004123,1.036431,1,True,6
2,WeightedEnsemble_L2,-3.998425,-2.075092,0.496564,0.174509,3.193284,0.016717,0.000399,0.264772,2,True,12
3,LightGBM,-4.074636,-2.232114,0.025479,0.035480,0.713540,0.025479,0.035480,0.713540,1,True,4
4,ExtraTreesMSE,-4.086322,-2.144460,0.224062,0.104672,0.497192,0.224062,0.104672,0.497192,1,True,7
5,RandomForestMSE,-4.087021,-2.255846,0.124471,0.104515,0.834154,0.124471,0.104515,0.834154,1,True,5
6,LightGBMLarge,-4.099543,-2.176688,0.048727,0.005880,0.350123,0.048727,0.005880,0.350123,1,True,11
7,XGBoost,-4.143115,-2.343701,0.043067,0.006015,0.341638,0.043067,0.006015,0.341638,1,True,9
8,NeuralNetFastAI,-4.266178,-2.845783,0.076772,0.010963,0.478356,0.076772,0.010963,0.478356,1,True,8
9,NeuralNetTorch,-4.861209,-2.745678,0.061740,0.011099,0.547665,0.061740,0.011099,0.547665,1,True,10
